In [0]:
from google.colab import drive
drive.mount('/content/drive')
!pip install sentencepiece

!git clone https://github.com/facebookresearch/fastText.git
!pip install /content/fastText

In [0]:
import fasttext

%tensorflow_version 1.x

from pathlib import Path
dirRoot=Path('/content/drive/My Drive/bert-based-text-generation/')
dirWordReplacement=dirRoot/"word_replacement"

import sys
sys.path.append(str(dirWordReplacement))
from run import run

import warnings
warnings.filterwarnings('ignore')
import subprocess
from IPython.display import clear_output
import numpy as np

In [0]:
seed_text="""

Hello!
This is a demo for BERT based text generation.

"""

mask_prob = 0.01 #@param {type:"slider", min:0.01, max:0.2, step:0.01}

In [8]:
inputText=[]
for line in seed_text.splitlines():
  line=line.strip()
  if len(line)==0: continue
  inputText.append(line)

print("Seed:")
print()
for line in inputText:
  print(line)

Seed:

Hello!
This is a demo for BERT based text generation.


In [9]:
iterSize=3

initRandState=None
fileRandState=None
fileFastText=None
kwargs={}
kwargs["mask_prob"]=mask_prob

modelType="auto"
if modelType=="auto":
  fileFastText=dirRoot/"data"/"lid.176.bin"
  model = fasttext.load_model(str(fileFastText))
  numPred=8
  langCandidate=("__label__en", "__label__ja")
  while True:
    langConfidence=model.predict("".join(inputText), k=numPred)
    langConfidence=dict(zip(*langConfidence))
    contains=False
    for lang in langCandidate:
      if lang in langConfidence:
        contains=True
        break
    if contains: break
    numPred*=2
  for lang in langCandidate:
    if lang not in langConfidence: langConfidence[lang]=0
  conf=np.array([langConfidence[lang] for lang in langCandidate])
  index=conf.argmax()
  lang=langCandidate[index]
  if lang=="__label__en":
    modelType="orig"
  elif lang=="__label__ja":
    modelType="ja-sp"
  print("lang =",lang,", model = ",modelType)

if modelType=="ja-sp":
  # DIR_BERT_JA_MODEL=Path("../bert-japanese/model")
  DIR_BERT_JA_MODEL=dirRoot/"bert-japanese/model"
  fileInitCheckpoint = DIR_BERT_JA_MODEL/"model.ckpt-1400000"
  fileVocab=DIR_BERT_JA_MODEL/"wiki-ja.vocab"
  fileModel=DIR_BERT_JA_MODEL/"wiki-ja.model"
  kwargs["model_file"]=fileModel
elif modelType=="orig":
  # DIR_BERT_MODEL=Path("../bert_model/cased_L-24_H-1024_A-16")
  DIR_BERT_MODEL=dirRoot/"bert_model/cased_L-24_H-1024_A-16"
  fileInitCheckpoint = DIR_BERT_MODEL/"bert_model.ckpt"
  fileVocab=DIR_BERT_MODEL/"vocab.txt"
  fileBertConfig=DIR_BERT_MODEL/"bert_config.json"
  kwargs["bert_config_file"]=fileBertConfig

tmpFiles=[]

argv=[]
argv.append(("--model_type", modelType))
argv.append(("--init_checkpoint", str(fileInitCheckpoint)))
argv.append(("--vocab_file", str(fileVocab)))
argv.append(("--iter_size", str(iterSize)))

for key,arg in kwargs.items():
  argv.append(("--"+key, str(arg)))

fileTmpInput=dirWordReplacement/"TmpInput.txt"
tmpFiles.append(fileTmpInput)
argv.append(("--input_file", str(fileTmpInput)))
with open(fileTmpInput, "w", encoding="utf8") as f:
  for text in inputText: print(text, file=f)

fileTmpOutput=dirWordReplacement/"TmpOutput.txt"
tmpFiles.append(fileTmpOutput)
argv.append(("--output_file", str(fileTmpOutput)))


lang = __label__en , model =  orig


In [10]:
command=["python", str(dirWordReplacement/"word_replacement.py"), *[key+"="+value for key,value in argv]]
p = subprocess.Popen(command, stdout = subprocess.PIPE, stderr = subprocess.STDOUT)

epoch=None
waiting=True
print("Seed:")
print()
for line in inputText: print(line)
print()
print("Computing.", end="")
for line in iter(p.stdout.readline,b''): #@see https://qiita.com/caprest/items/0245a16825789b0263ad
  line=line.rstrip().decode("utf8")
  if "[Generated text begin]" in line:
    clear_output(wait=True)
    waiting=False
    print("Epoch:", epoch)
    print()
    continue
  elif line.startswith("Generated:"):
    print(line[len("Generated:"):].strip())
  elif line.startswith("Iteration"):
    epoch=int(line[len("Iteration:"):].strip())
  if "[Generated text end]" in line and epoch<iterSize-1:
    waiting=True
    print()
    print("Computing.", end="")
  elif waiting and (epoch is None or epoch<iterSize-1):
    print(".", end="")

Epoch: 2

Go.
This is a demo of BERT for text generation;
